# Select and distribute additional Auditory Percetion articles for phrase annotation

## Goal is deeper annotation.
- A good number of annotations on this topic.
- Phrase based annotation so we can be most flexible and assess at higher levels.

## Secondary goal, more cross-validation.
- 1/2 of new articles will be annotated by all.
- 1/2 of new articles will be annotated by only 1 annotator.

In [21]:
dest_dir = '/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/20160205_rdoc_project'
%mkdir {dest_dir}
%cd {dest_dir}

mkdir: /Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/20160205_rdoc_project: File exists
/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/20160205_rdoc_project


### Get a list of all our previous pubmed ids used for annotation (irrespective of Auditory perception)

In [22]:
from __future__ import print_function
import glob
import os
import re
# annotated and processed previously
prev_annotated_pdfs_dir = '/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/all_pdfs_annotated_pmid_names/*.pdf'
pdfs = glob.glob(prev_annotated_pdfs_dir)
pdfs = [os.path.basename(pdf) for pdf in pdfs]

# and those from round 4 annotations
prev_annotated_pdfs_dir2 = '/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/20160122_rdoc_project/*.pdf'
pdfs2 = glob.glob(prev_annotated_pdfs_dir2)

pdfs.extend([os.path.basename(pdf) for pdf in pdfs2])
print('{} total PDFS'.format(len(pdfs)))

327 total PDFS


In [23]:
pattern = '([0-9]{6,9})'
p = re.compile(pattern)
pmids = [p.search(pdf).group() for pdf in pdfs if p.search(pdf)]
print('{} Total exisiting PMIDS'.format(len(pmids)))
pmids = set(pmids)
print('{} Unique exisiting PMIDS'.format(len(pmids)))
#print(pmids)
prev_pmids = pmids

324 Total exisiting PMIDS
229 Unique exisiting PMIDS


229 unique existing pmids:

['22686386', '23744445', '22715197', '23055094', '23929944', '24806675', '22575329', '24930577', '25107317', '22776995', '24870123', '24388670', '21849230', '24933663', '20497902', '25734385', '20532489', '23555220', '24770625', '25834059', '21854953', '23904684', '22378876', '21206465', '23647728', '23127585', '20410871', '25142564', '21203376', '21954087', '22805501', '22379245', '25261920', '22505867', '24512610', '22239924', '21463060', '21699821', '24453310', '23504052', '25160677', '22088577', '21278378', '21426626', '22285891', '22664396', '24333745', '22968207', '25051268', '22438994', '20955866', '24964082', '22233352', '23190433', '21826029', '23934417', '22911829', '25280468', '23074247', '25142762', '22169884', '21613467', '24529045', '25017671', '22832959', '21276977', '24466064', '22447249', '22314045', '25521352', '22294809', '24651580', '24015304', '23990240', '24933724', '24126129', '21957257', '22474609', '20584721', '22048839', '24156344', '25348125', '24259275', '24899756', '21034683', '21050743', '23503620', '25788679', '23709163', '21259270', '21477924', '23299717', '21909371', '22004270', '21397252', '21419826', '25759762', '24252875', '24293773', '24235891', '24231418', '24116095', '22146934', '22131608', '20685988', '22986355', '22414937', '24359877', '23928891', '22496862', '23060019', '25036222', '25424865', '20695690', '25036160', '24804717', '23786695', '25749431', '23326604', '22215928', '20049632', '21531705', '20842164', '20663220', '23770566', '21898707', '23899724', '23622762', '24740391', '25126029', '23088207', '25348131', '25456277', '22286850', '23940642', '25061837', '24705497', '25773639', '21623612', '20525011', '25197810', '25126038', '25154749', '24618591', '24470693', '21118712', '21179552', '24980898', '23957953', '22315106', '24571111', '22043127', '23707592', '24101292', '24285346', '20620104', '24511281', '21077571', '23954763', '22467988', '24376698', '22470524', '23810448', '22035386', '25898427', '22379238', '22379239', '23452958', '24266644', '24426818', '23941878', '21626350', '24260331', '25325584', '21278907', '23646134', '25136085', '21675365', '23731439', '22163262', '24457211', '22998925', '24505292', '23143607', '25774613', '23083918', '21120131', '20939652', '25740534', '24214921', '23643925', '21440905', '25883640', '20634711', '22377810', '25913552', '23997704', '24023823', '21731598', '24412227', '21376761', '24333377', '21550590', '24725811', '21319926', '24045586', '21335015', '25258728', '22215925', '22119526', '24127147', '23193115', '21280045', '25024660', '21677894', '21816115', '20815182', '20661292', '21886801', '20615239', '23558179', '25756280', '20857862', '21392554', '22665872', '25581922', '20450941', '24125792', '22959616']

## Get new lists of pmids specific to topic

In [24]:
#from __future__ import print_function
from Bio import Entrez
from subprocess import check_call
from shutil import copy2
#import glob
import time
import imp
import os
url2p = imp.load_source('Url2PubmedPmcPdf', '/Users/ccarey/Documents/Projects/NAMI/rdoc/scripts/Url2PubmedPmcPdf.py')
Entrez.email = "charlieccarey@gmail.com"

def narrow_id_list(found_ids, omit_ids):
    found_but_omit = list(set(found_ids) & set(omit_ids))
    found_and_keep = list(set(found_ids) - set(omit_ids))
    print('Removed this many ids: {}'.format(len(found_but_omit)))
    return(found_and_keep)

def pubmed_central_search_to_pubmed_id(search_string, retmax=20):
    # verify how many records match
    handle = Entrez.egquery(term=search_string)
    record = Entrez.read(handle)
    # maybe useful if we are dealing with 100s of ids and don't want to overwhelm server?
    for row in record["eGQueryResult"]:
        if row["DbName"] == "pubmed":
            print(row["Count"])
    # fetch the ids for those records
    handle = Entrez.esearch(db="pubmed", retmax=retmax, term=search_string)
    record = Entrez.read(handle)
    pubmed_ids = record["IdList"]
    return(pubmed_ids)

def fetch_pdfs(pubmed_ids, stub_name):
    u = url2p.Url2PubmedPmcPdf(pubmed_ids)
    urls = u.get_urls()
    found = []
    for url in urls:
        if url['url'] is not None:
            cmd = 'curl -L {} -o {}.pdf'.format(url['url'], stub_name + url['pubmed'])
            # print(cmd)
            check_call(cmd, shell = True)
            time.sleep(10)
            found.append(url['pubmed'])
    return(found)

# def copy_pdf_append_initials(initials):
#     pdfs = glob.glob('*.pdf')
#     for i in initials:
#         os.mkdir(i)
#         for p in pdfs:
#             pi = p.replace('.pdf', '_' + i + '.pdf')
#             copy2(p, os.path.join(i, pi))

def search_and_summarize(search_name, query, omit_ids):
    ids = pubmed_central_search_to_pubmed_id(query, retmax=1000000)
    new_ids = narrow_id_list(ids, omit_ids)
    print('{} search of pubmed found {} ids of which {} are new'.format(search_name, len(ids), len(new_ids)))
    return(new_ids)

### Desire to send out more articles. 30 in common to all, 30 unique to all
 - 30 common to all + 30 unique per annotator * 4 annotators
 - 120 articles total desired.

In [43]:
# again, these are from preliminary work by Aurelien etc.
AP00 = '"Auditory Perception"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND Humans[Mesh])'
AP01 = '"Auditory Perception"[All Fields] AND "mismatch negativity "[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'
AP02 = '"Auditory Perception"[All Fields] AND (("social control, formal"[MeSH Terms] OR ("social"[All Fields] AND "control"[All Fields] AND "formal"[All Fields]) OR "formal social control"[All Fields] OR "regulation"[All Fields]) AND ("haemodynamic"[All Fields] OR "hemodynamics"[MeSH Terms] OR "hemodynamics"[All Fields] OR "hemodynamic"[All Fields]) AND components[All Fields] AND sensory[All Fields] AND response[All Fields] AND ("substance-related disorders"[MeSH Terms] OR ("substance-related"[All Fields] AND "disorders"[All Fields]) OR "substance-related disorders"[All Fields] OR "habituation"[All Fields])) AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'
AP03 = '"Auditory Perception"[All Fields] AND "hemodynamic"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'
AP04 = '"Auditory Perception"[All Fields] AND "spatial localization"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'
AP05 = '"Auditory Perception"[All Fields] AND (Sensory[All Fields] AND ("evoked potentials"[MeSH Terms] OR ("evoked"[All Fields] AND "potentials"[All Fields]) OR "evoked potentials"[All Fields] OR "erp"[All Fields])) AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'
AP06 = '"Auditory Perception"[All Fields] AND "auditory hallucinations"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'
AP07 = '"Auditory Perception"[All Fields] AND "hyperacusis"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'
AP08 = '"Auditory Perception"[All Fields] AND "McGurk"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'
AP09 = '"Auditory Perception"[All Fields] AND (auditory[All Fields] AND scene[All Fields] AND ("perception"[MeSH Terms] OR "perception"[All Fields])) AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'
AP10 = '"Auditory Perception"[All Fields] AND "auditory masking"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'
AP11 = '"Auditory Perception"[All Fields] AND "Gating"[All Fields] AND ("loattrfree full text"[sb] AND "2010/06/24"[PDat] : "2015/06/22"[PDat] AND "humans"[MeSH Terms])'

In [48]:
AP00_ids = search_and_summarize(search_name='AP00', query=AP00, omit_ids=prev_pmids)
AP01_ids = search_and_summarize(search_name='AP01', query=AP01, omit_ids=prev_pmids)
AP02_ids = search_and_summarize(search_name='AP02', query=AP02, omit_ids=prev_pmids)
AP03_ids = search_and_summarize(search_name='AP03', query=AP03, omit_ids=prev_pmids)
AP04_ids = search_and_summarize(search_name='AP04', query=AP04, omit_ids=prev_pmids)
AP05_ids = search_and_summarize(search_name='AP05', query=AP05, omit_ids=prev_pmids)
AP06_ids = search_and_summarize(search_name='AP06', query=AP06, omit_ids=prev_pmids)
AP07_ids = search_and_summarize(search_name='AP07', query=AP07, omit_ids=prev_pmids)
AP08_ids = search_and_summarize(search_name='AP08', query=AP08, omit_ids=prev_pmids)
AP09_ids = search_and_summarize(search_name='AP09', query=AP09, omit_ids=prev_pmids)
AP10_ids = search_and_summarize(search_name='AP10', query=AP10, omit_ids=prev_pmids)
AP11_ids = search_and_summarize(search_name='AP11', query=AP11, omit_ids=prev_pmids)

1449
Removed this many ids: 17
AP00 search of pubmed found 1449 ids of which 1432 are new

56
Removed this many ids: 3
AP01 search of pubmed found 56 ids of which 53 are new

0
Removed this many ids: 0
AP02 search of pubmed found 0 ids of which 0 are new

9
Removed this many ids: 2
AP03 search of pubmed found 9 ids of which 7 are new

3
Removed this many ids: 1
AP04 search of pubmed found 3 ids of which 2 are new

70
Removed this many ids: 1
AP05 search of pubmed found 70 ids of which 69 are new

3
Removed this many ids: 1
AP06 search of pubmed found 3 ids of which 2 are new

6
Removed this many ids: 3
AP07 search of pubmed found 6 ids of which 3 are new

10
Removed this many ids: 2
AP08 search of pubmed found 10 ids of which 8 are new

50
Removed this many ids: 0
AP09 search of pubmed found 50 ids of which 50 are new

1
Removed this many ids: 0
AP10 search of pubmed found 1 ids of which 1 are new

17
Removed this many ids: 2
AP11 search of pubmed found 17 ids of which 15 are new


1449
Removed this many ids: 17
AP00 search of pubmed found 1449 ids of which 1432 are new

56
Removed this many ids: 3
AP01 search of pubmed found 56 ids of which 53 are new

0
Removed this many ids: 0
AP02 search of pubmed found 0 ids of which 0 are new

9
Removed this many ids: 2
AP03 search of pubmed found 9 ids of which 7 are new

3
Removed this many ids: 1
AP04 search of pubmed found 3 ids of which 2 are new

70
Removed this many ids: 1
AP05 search of pubmed found 70 ids of which 69 are new

3
Removed this many ids: 1
AP06 search of pubmed found 3 ids of which 2 are new

6
Removed this many ids: 3
AP07 search of pubmed found 6 ids of which 3 are new

10
Removed this many ids: 2
AP08 search of pubmed found 10 ids of which 8 are new

50
Removed this many ids: 0
AP09 search of pubmed found 50 ids of which 50 are new

1
Removed this many ids: 0
AP10 search of pubmed found 1 ids of which 1 are new

17
Removed this many ids: 2
AP11 search of pubmed found 17 ids of which 15 are new

In [49]:
import collections

# total found across all searches
ids = [AP00_ids, AP01_ids, AP02_ids, AP03_ids, AP04_ids, AP05_ids, AP06_ids, AP07_ids, AP08_ids, AP09_ids, AP10_ids, AP11_ids]
ids = [j for i in ids for j in i]
print('total new pmids found across all searches : {}'.format(len(ids)))

# excluding most general search.
ids = [AP01_ids, AP02_ids, AP03_ids, AP04_ids, AP05_ids, AP06_ids, AP07_ids, AP08_ids, AP09_ids, AP10_ids, AP11_ids]
ids = [j for i in ids for j in i]
dups = [item for item, count in collections.Counter(ids).items() if count > 1]
print('duplicate pmids found across all searches : {}'.format(dups))

ids = [AP01_ids, AP02_ids, AP03_ids, AP04_ids, AP05_ids, AP06_ids, AP07_ids, AP08_ids, AP09_ids, AP10_ids, AP11_ids]
ids = [set(id_list) for id_list in ids]
ids = set.union(*ids)
print('unique new pmids found across all searches : {}'.format(len(ids)))

total new pmids found across all searches : 1642
duplicate pmids found across all searches : ['24771006', '25259525', '21261633', '24548430', '23308266', '21823798', '20578033', '21625011', '21500313', '22508089', '23242351', '23076102', '22464943', '22815876', '22786953', '24158725', '25379456', '21368051', '24956028', '23015429', '24336606', '25447378', '20633569']
unique new pmids found across all searches : 187


### Found plenty new articles

- total new pmids found across all but most general searches : 
  - 187
- But actually retrieved and unique across Audtory Perception for annotation (see below): 
  - 146


### Fetch the pdfs for each search term.

In [50]:
%pwd

u'/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/20160205_rdoc_project'

In [ ]:
# 5th batch, Auditory perception, subterm...

batch = '05'

fetch_pdfs(AP01_ids, batch + '_AP01_')
#fetch_pdfs(AP02_ids, batch + '_AP02_') # none available
fetch_pdfs(AP03_ids, batch + '_AP03_')
fetch_pdfs(AP04_ids, batch + '_AP04_')
fetch_pdfs(AP05_ids, batch + '_AP05_')
fetch_pdfs(AP06_ids, batch + '_AP06_')
fetch_pdfs(AP07_ids, batch + '_AP07_')
fetch_pdfs(AP08_ids, batch + '_AP08_')
fetch_pdfs(AP09_ids, batch + '_AP09_')
fetch_pdfs(AP10_ids, batch + '_AP10_')
fetch_pdfs(AP11_ids, batch + '_AP11_')

I then listed the possible duplicate IDs and removed the later occuring duplicates. i.e. if ID occurred in AP01 and AP04 I only kept the AP01 version.

In [54]:
all_found = glob.glob( './*.pdf')
len(all_found)

146

In [ ]:
import collections

def report_found(search_ids, glob_pattern, id_pattern):
    '''Report the pmids found within the search set of files.'''
    files = glob.glob(glob_pattern)
    p = re.compile('[0-9]{8,8}')
    found = [p.search(pdf).group() for pdf in files if p.search(pdf)]
    not_found = set(search_ids).difference(found)
    print()
    #print('{} searched = {}'.format(len(search_ids), search_ids))
    #print('{} found = {}'.format(len(found), found))
    print('{} {} searched.'.format(len(search_ids), glob_pattern))
    print('{} {} found.'.format(len(found), glob_pattern))
    print('{} {} not_found = {}'.format(len(not_found), glob_pattern, not_found))
    return(found)

all_found_d = {}

all_found_d['AP01'] = report_found(AP01_ids, glob_pattern='05_AP01_*', id_pattern='[0-9]{8,8}')
all_found_d['AP03'] = report_found(AP03_ids, glob_pattern='05_AP03_*', id_pattern='[0-9]{8,8}')
all_found_d['AP04'] = report_found(AP04_ids, glob_pattern='05_AP04_*', id_pattern='[0-9]{8,8}')
all_found_d['AP05'] = report_found(AP05_ids, glob_pattern='05_AP05_*', id_pattern='[0-9]{8,8}')
all_found_d['AP06'] = report_found(AP06_ids, glob_pattern='05_AP06_*', id_pattern='[0-9]{8,8}')
all_found_d['AP07'] = report_found(AP07_ids, glob_pattern='05_AP07_*', id_pattern='[0-9]{8,8}')
all_found_d['AP08'] = report_found(AP08_ids, glob_pattern='05_AP08_*', id_pattern='[0-9]{8,8}')
all_found_d['AP09'] = report_found(AP09_ids, glob_pattern='05_AP09_*', id_pattern='[0-9]{8,8}')
all_found_d['AP10'] = report_found(AP10_ids, glob_pattern='05_AP10_*', id_pattern='[0-9]{8,8}')
all_found_d['AP11'] = report_found(AP11_ids, glob_pattern='05_AP11_*', id_pattern='[0-9]{8,8}')

# print()
# print('------- Reporting Found: ----------')
# print()
# print('{} Total found'.format(sum([len(v) for v in all_found_d.values()])))
# print()
all_found_d = collections.OrderedDict(sorted(all_found_d.items()))
# for k,v in all_found_d.iteritems():
#     print('{}: {}'.format(k, v))

### Reported found or not found for batch 4
53 05_AP01_* searched.
38 05_AP01_* found.
15 05_AP01_* not_found = set(['23242351', '21248117', '26142650', '21624926', '22279229', '25447378', '25173955', '23076102', '23042732', '24992584', '23015429', '21566197', '24336606', '26223716', '23148350'])

7 05_AP03_* searched.
4 05_AP03_* found.
3 05_AP03_* not_found = set(['20861363', '23804092', '24401714'])

2 05_AP04_* searched.
2 05_AP04_* found.
0 05_AP04_* not_found = set([])

69 05_AP05_* searched.
42 05_AP05_* found.
27 05_AP05_* not_found = set(['23242351', '22171057', '24525177', '23042734', '23678108', '21500313', '22790547', '22815876', '22786953', '23954727', '25632143', '25379456', '23308266', '21823798', '26041921', '21625011', '21325518', '21368051', '20578033', '21068187', '23015429', '24336606', '25653372', '22508089', '21893681', '24055864', '21613485'])

2 05_AP06_* searched.
1 05_AP06_* found.
1 05_AP06_* not_found = set(['24360035'])

3 05_AP07_* searched.
1 05_AP07_* found.
2 05_AP07_* not_found = set(['23876286', '25387539'])

8 05_AP08_* searched.
5 05_AP08_* found.
3 05_AP08_* not_found = set(['23810539', '25447378', '21625011'])

50 05_AP09_* searched.
41 05_AP09_* found.
9 05_AP09_* not_found = set(['24158725', '24956028', '25948273', '22090489', '24771006', '24548430', '23076102', '21261633', '25259525'])

1 05_AP10_* searched.
1 05_AP10_* found.
0 05_AP10_* not_found = set([])

15 05_AP11_* searched.
11 05_AP11_* found.
4 05_AP11_* not_found = set(['23035108', '20633569', '22508089', '22464943'])

------- Reporting Found: ----------

146 Total found

AP01: ['19929331', '20578033', '20665718', '20929535', '21368051', '21483666', '21750713', '21808660', '21823798', '22163029', '22213909', '22221004', '22551948', '22570723', '22815876', '22916282', '23028971', '23131615', '23241212', '23308266', '23585888', '23617597', '23708059', '23715097', '23825422', '23850664', '23886958', '23920129', '24143195', '24158725', '24366694', '24475052', '24548430', '24771006', '25178752', '25231619', '25342520', '25379456']

AP03: ['21159322', '21500313', '22496909', '23777481']

AP04: ['22786953', '25710328']

AP05: ['20146608', '20493828', '20598152', '20633569', '21233780', '21261633', '21305666', '21380858', '21807011', '21958655', '22367585', '22464943', '22547804', '22592306', '22628458', '22735387', '22773777', '22885999', '23071654', '23145143', '23251704', '23281832', '23301775', '23316957', '23326548', '23616340', '23624493', '23647558', '23664703', '23827717', '23935931', '23988583', '24072639', '24119225', '24314010', '24423729', '24695717', '24956028', '25245785', '25259525', '25535356', '25948269']

AP06: ['22803512']

AP07: ['25231612']

AP08: ['21909378', '22390292', '23664001', '24974346', '24996043']

AP09: ['20541597', '20826671', '20844143', '20975559', '21196054', '21209201', '21355664', '21387016', '21428515', '21945789', '22036957', '22280585', '22371616', '22371619', '22371621', '22612172', '22753470', '22829899', '22844509', '23029426', '23145699', '23423817', '23516340', '23527271', '23691185', '23825404', '23926291', '24003112', '24052177', '24239869', '24475030', '24478375', '24681354', '24711409', '24788808', '24821552', '24841996', '25433224', '25654748', '25659464', '25726262']

AP10: ['22240459']

AP11: ['20649227', '21131368', '22087275', '22384211', '22896044', '23716244', '24298171', '24736181', '24801767', '25024207', '25544613']

#### Confirming we removed all duplicates

In [72]:
all_found = [v for k,v in all_found_d.items()]
all_found = [item for sublist in all_found for item in sublist]
print(len(all_found) == len(set(all_found)))
print(len(all_found))

True
146


In [73]:
# 32 for everyone.
# 38 unique to each of 3 annotators.
import random
files = glob.glob('05_AP*.pdf')
idxs = range(len(files))
random.shuffle(idxs)

everyone = [files[idx] for idx in idxs[0:32]]
mk = [files[idx] for idx in idxs[32:70]]
cc = [files[idx] for idx in idxs[70:108]]
tc = [files[idx] for idx in idxs[108:146]]

In [ ]:
def copy_to_annotator(files, annotator, suffix):
    %mkdir {annotator}
    %mkdir {annotator}/annotated
    %mkdir {annotator}/irrelevant
    for f in files:
        f_dest = f.split('.')[0] + suffix
        %cp {f} ./{annotator}/{f_dest}

copy_to_annotator(mk, 'mk_', '_mk.pdf')
copy_to_annotator(everyone, 'mk_', '_mk.pdf')

# oops... gave tara exact same set as mk
copy_to_annotator(mk, 'tc_', '_tc.pdf')
copy_to_annotator(everyone, 'tc_', '_tc.pdf')

# copy_to_annotator(cc, 'cc_', '_cc.pdf')
# copy_to_annotator(everyone, 'cc_', '_cc.pdf')

## Add the new abstracts to our medic database
Insert fails if any are already in db.
Update will overwrite previous records.

In [ ]:
print(len(all_found))
print(all_found)

146
['19929331', '20578033', '20665718', '20929535', '21368051', '21483666', '21750713', '21808660', '21823798', '22163029', '22213909', '22221004', '22551948', '22570723', '22815876', '22916282', '23028971', '23131615', '23241212', '23308266', '23585888', '23617597', '23708059', '23715097', '23825422', '23850664', '23886958', '23920129', '24143195', '24158725', '24366694', '24475052', '24548430', '24771006', '25178752', '25231619', '25342520', '25379456', '21159322', '21500313', '22496909', '23777481', '22786953', '25710328', '20146608', '20493828', '20598152', '20633569', '21233780', '21261633', '21305666', '21380858', '21807011', '21958655', '22367585', '22464943', '22547804', '22592306', '22628458', '22735387', '22773777', '22885999', '23071654', '23145143', '23251704', '23281832', '23301775', '23316957', '23326548', '23616340', '23624493', '23647558', '23664703', '23827717', '23935931', '23988583', '24072639', '24119225', '24314010', '24423729', '24695717', '24956028', '25245785', '25259525', '25535356', '25948269', '22803512', '25231612', '21909378', '22390292', '23664001', '24974346', '24996043', '20541597', '20826671', '20844143', '20975559', '21196054', '21209201', '21355664', '21387016', '21428515', '21945789', '22036957', '22280585', '22371616', '22371619', '22371621', '22612172', '22753470', '22829899', '22844509', '23029426', '23145699', '23423817', '23516340', '23527271', '23691185', '23825404', '23926291', '24003112', '24052177', '24239869', '24475030', '24478375', '24681354', '24711409', '24788808', '24821552', '24841996', '25433224', '25654748', '25659464', '25726262', '22240459', '20649227', '21131368', '22087275', '22384211', '22896044', '23716244', '24298171', '24736181', '24801767', '25024207', '25544613']


In [83]:
cmd = ' '.join(all_found)

In [84]:
!medic update {cmd} 2> /dev/null

In [ ]:
!medic --format tsv write ALL 2> /dev/null | cut -f1 | wc # 3359

## Save these new ids to a file

In [86]:
batch_05_fname = '/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/task_data_temp/batch_05_AP_pmids'
with open(batch_05_fname, 'wb') as f:
    for pmid in all_found:
        f.write(pmid + '\n')

In [90]:
# confirm we have 146 new abstracts
!medic --format tsv write --pmid-list {batch_05_fname} 2> /dev/null | cut -f1 | wc

     146     146    1314


# Noted mistake, accidentally gave entirely same set to everyone.
desired:
- 32 for everyone.
- 38 unique to each of 3 annotators.

- don't change mk
- change Taras (but keep those she has already done as part of the 'common' set.)
- change Chucks
- add in common set to Janet


In [172]:

import re
import os
import glob
import collections
import random
random.seed(0)

pattern = '([0-9]{8,8})'
p = re.compile(pattern)

In [173]:
random.seed(0)
pmids = ['19929331', '20578033', '20665718', '20929535', '21368051', '21483666', '21750713', '21808660', '21823798', '22163029', '22213909', '22221004', '22551948', '22570723', '22815876', '22916282', '23028971', '23131615', '23241212', '23308266', '23585888', '23617597', '23708059', '23715097', '23825422', '23850664', '23886958', '23920129', '24143195', '24158725', '24366694', '24475052', '24548430', '24771006', '25178752', '25231619', '25342520', '25379456', '21159322', '21500313', '22496909', '23777481', '22786953', '25710328', '20146608', '20493828', '20598152', '20633569', '21233780', '21261633', '21305666', '21380858', '21807011', '21958655', '22367585', '22464943', '22547804', '22592306', '22628458', '22735387', '22773777', '22885999', '23071654', '23145143', '23251704', '23281832', '23301775', '23316957', '23326548', '23616340', '23624493', '23647558', '23664703', '23827717', '23935931', '23988583', '24072639', '24119225', '24314010', '24423729', '24695717', '24956028', '25245785', '25259525', '25535356', '25948269', '22803512', '25231612', '21909378', '22390292', '23664001', '24974346', '24996043', '20541597', '20826671', '20844143', '20975559', '21196054', '21209201', '21355664', '21387016', '21428515', '21945789', '22036957', '22280585', '22371616', '22371619', '22371621', '22612172', '22753470', '22829899', '22844509', '23029426', '23145699', '23423817', '23516340', '23527271', '23691185', '23825404', '23926291', '24003112', '24052177', '24239869', '24475030', '24478375', '24681354', '24711409', '24788808', '24821552', '24841996', '25433224', '25654748', '25659464', '25726262', '22240459', '20649227', '21131368', '22087275', '22384211', '22896044', '23716244', '24298171', '24736181', '24801767', '25024207', '25544613']

tc_done = glob.glob('/Users/ccarey/Dropbox/20150828_rdoc_project/tc_/annotated/05_AP*.pdf')
tc_done = [os.path.basename(tc) for tc in tc_done]
tc_done_ids = [p.search(pdf).group() for pdf in tc_done if p.search(pdf)]
tc_all = glob.glob('/Users/ccarey/Dropbox/20150828_rdoc_project/tc_/05_AP*.pdf')
tc_all = [os.path.basename(tc) for tc in tc_all]
tc_all_ids = [p.search(pdf).group() for pdf in tc_all if p.search(pdf)]
# Tara did 11 already
print(len(tc_done_ids))
tc_not_done = [tc for tc in tc_all_ids if tc not in tc_done_ids]
random.shuffle(tc_not_done)
tc_more_common = tc_not_done[0:21]
tc_all_common = tc_more_common + tc_done_ids
everyone = tc_all_common
print(everyone)

11
['23071654', '23850664', '24801767', '23647558', '23777481', '23585888', '22464943', '24788808', '23827717', '22496909', '22803512', '23886958', '23617597', '21387016', '23825422', '22829899', '21909378', '22240459', '22896044', '21355664', '22367585', '19929331', '20578033', '20929535', '21368051', '21483666', '21808660', '22221004', '22551948', '22570723', '23028971', '23131615']


In [174]:
everyone = ['23071654', '23850664', '24801767', '23647558', '23777481', '23585888', '22464943', '24788808', '23827717', '22496909', '22803512', '23886958', '23617597', '21387016', '23825422', '22829899', '21909378', '22240459', '22896044', '21355664', '22367585', '19929331', '20578033', '20929535', '21368051', '21483666', '21808660', '22221004', '22551948', '22570723', '23028971', '23131615']

In [168]:
# fixing TC
%pwd

u'/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/20160205_rdoc_project'

In [175]:
def copy_to_annotator(files, annotator, suffix):
    %mkdir {annotator}
    %mkdir {annotator}/annotated
    %mkdir {annotator}/irrelevant
    for f in files:
        f_dest = f.split('.')[0] + suffix
        %cp {f} ./{annotator}/{f_dest}

print(everyone)
print(len(everyone))
everyone_files = [glob.glob('*' + e + '.pdf') for e in everyone]

['23071654', '23850664', '24801767', '23647558', '23777481', '23585888', '22464943', '24788808', '23827717', '22496909', '22803512', '23886958', '23617597', '21387016', '23825422', '22829899', '21909378', '22240459', '22896044', '21355664', '22367585', '19929331', '20578033', '20929535', '21368051', '21483666', '21808660', '22221004', '22551948', '22570723', '23028971', '23131615']
32


In [179]:
# pmids # all pmids for batch 05
mk_files = glob.glob('/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/20160205_rdoc_project/out_20160205/mk_/*.pdf')
mk_files = [os.path.basename(mk) for mk in mk_files]
mk_ids = [p.search(pdf).group() for pdf in mk_files if p.search(pdf)]
to_be_assigned = [pmid for pmid in pmids if pmid not in mk_ids and pmid not in everyone]

In [182]:
len(to_be_assigned)
random.shuffle(to_be_assigned)

76 unassigned pmids, assign 38 to TC, assign 38 to CC
to_be_assigned

In [185]:
to_be_assigned_files = [glob.glob('*' + t + '.pdf') for t in to_be_assigned]

In [186]:
# flatten list of list (each inner list was a single pdf)
everyone_files = [item for e in everyone_files for item in e]
to_be_assigned_files = [item for t in to_be_assigned_files for item in t]

In [161]:
to_be_assigned_files = [item for t in to_be_assigned_files for item in t]

In [190]:
tc_uniq = to_be_assigned_files[0:38]
cc_uniq = to_be_assigned_files[38:76]

In [191]:
copy_to_annotator(everyone_files, 'tc_', '_tc.pdf')
copy_to_annotator(tc_uniq, 'tc_', '_tc.pdf')

mkdir: tc_: File exists
mkdir: tc_/annotated: File exists
mkdir: tc_/irrelevant: File exists
mkdir: tc_: File exists
mkdir: tc_/annotated: File exists
mkdir: tc_/irrelevant: File exists


In [192]:
copy_to_annotator(everyone_files, 'cc_', '_cc.pdf')
copy_to_annotator(cc_uniq, 'cc_', '_cc.pdf')

copy_to_annotator(everyone_files, 'jl_', '_jl.pdf')

mkdir: cc_: File exists
mkdir: cc_/annotated: File exists
mkdir: cc_/irrelevant: File exists


13
['23193115', '23941878', '19929331', '20578033', '20929535', '21368051', '21483666', '21808660', '22221004', '22551948', '22570723', '23028971', '23131615']
32


In [20]:
dest_dir = '/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/20160205_rdoc_project'
%cd {dest_dir}
pdfs = glob.glob('out_20160205/*/*.pdf')
pdfs = [os.path.basename(p) for p in pdfs]
pattern = '([0-9]{8,8})'
p = re.compile(pattern)
temp_ids = [p.search(pdf).group() for pdf in pdfs if p.search(pdf)]
counter=collections.Counter(temp_ids)
# noting mistake, accidentally gave all the same to everyone.
counter

/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/20160205_rdoc_project


Counter({'19929331': 3,
         '20146608': 3,
         '20541597': 3,
         '20578033': 3,
         '20633569': 3,
         '20929535': 3,
         '21131368': 3,
         '21159322': 3,
         '21196054': 3,
         '21209201': 3,
         '21233780': 3,
         '21261633': 3,
         '21355664': 3,
         '21368051': 3,
         '21387016': 3,
         '21428515': 3,
         '21483666': 3,
         '21808660': 3,
         '21909378': 3,
         '21958655': 3,
         '22087275': 3,
         '22221004': 3,
         '22240459': 3,
         '22367585': 3,
         '22371619': 3,
         '22371621': 3,
         '22390292': 3,
         '22464943': 3,
         '22496909': 3,
         '22547804': 3,
         '22551948': 3,
         '22570723': 3,
         '22628458': 3,
         '22803512': 3,
         '22829899': 3,
         '22885999': 3,
         '22896044': 3,
         '23028971': 3,
         '23071654': 3,
         '23131615': 3,
         '23241212': 3,
         '233017